# Knowledge Graph

## Einführung
Im 4. Assignment wird ein Knowledge Graph implementiert, und die bisherigen RDF-Daten mit neuen Informationen angereichert, die über externe Links und Quellen abgefragt werden. Außerdem wird die von uns erstellte lokale Suchmaschine, die wir in Assignment 2 entwickelt haben, erweitert - hierzu wird das von uns entwickelte Ontologie-Modell verwendet, welches in Assignment 3 implementiert wurde.
Zum Schluss werden mithilfe von SPARQL semantische Abfragen erstellt, durch die wir Informationen über den Damen-Volleyballkader erhalten können.

## Arbeitsaufteilung
* Cesar Laura:
* Dilly Julian: SPARQL und Triple Storage
* Ecker Annina: 1. Integration Onthology with Search Engine


### Installation und Importieren der benötigten Libraries

In [12]:
#!pip install -r requirements.txt

In [1]:
import pandas as pd
import text_processing as tp
import tfidf_search as ts
import ontology_graph as og

[nltk_data] Downloading package punkt to /Users/annina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Laden der Ontologie - Erstellen des RDF Graphs

In [2]:
volleyball_graph = og.create_volleyball_ontology()

### 1. Integration der Ontologie in die lokale Suchmaschine

In [3]:
spielerinnen_csv = pd.read_csv("./Spielerinnendaten.csv")

# text_processing + Ontologie einfügen
inverted_index = tp.build_inverted_index(spielerinnen_csv.values)

#### 1.1 Änderung 
Bei der Implementierung des bisherigen TFIDF-Scores hat sich gezeigt, dass `cm` als eigener Token interpretiert wurde, und nicht als Teil der Information zur Körpergröße:
<details>
  <summary>Screenshot: Available Tokens, Erstversion</summary>
  
  ![title](available_tokens.png)
  
</details>

Daher wurde die Logik für die Funktion `clean_text`angepasst, wo mithilfe von RegEx Einheit und Zahl durch einen `_` kombiniert werden.

<details>
    <summary>Code: Before & After</summary>
    
![title](height_token_solution.png)
  
</details>

**Diese Änderung wurde jedoch wieder rückgängig gemacht, da im weiteren Verlauf der Implementierung bessere Suchanfragen mittels SPARQL erstellt werden sollen.**

<details>
    <summary>Suchanfrage mit geänderter RegEx-Abfrage (wurde wieder rückgängig gemacht!):</summary>
    
![title](suche_w_regex.png)
  
</details>

---

In [4]:
tfidf_df = ts.calculate_tfidf(spielerinnen_csv.values, inverted_index)

# Check aller Tokens:
# Zeige die existierenden Tokens in der TF-IDF Tabelle
print(f"Available tokens in the tfidf_df Dataframe:{tfidf_df.columns}")

Available tokens in the tfidf_df Dataframe:Index(['1', '10', '11', '13', '14', '15', '166', '17', '171', '173', '174',
       '175', '179', '18', '180', '183', '185', '186', '188', '19', '2', '21',
       '22', '27', '28', '3', 'anna', 'aufspiel', 'aut', 'außenangriff',
       'carmen', 'chrtianska', 'cm', 'dana', 'diagonal', 'ehrhart', 'eva',
       'fitz', 'gaertner', 'hinteregger', 'holzinger', 'huber', 'jana',
       'janka', 'julia', 'kora', 'lelia', 'leonie', 'libero', 'lina', 'marina',
       'mittelblock', 'monika', 'nesimovic', 'nicole', 'nina', 'oberhauser',
       'raab', 'schaberl', 'schmit', 'stabentheiner', 'tamina', 'trunner',
       'ursula', 'verena'],
      dtype='object')


---

In [5]:
# Suchabfrage: Test 1
# Check der AND-Bedingung
ts.search(tfidf_df, "Diagonal and 185 and cm", spielerinnen_csv)

volleyball_graph = og.create_volleyball_ontology()

          Name Position Groesse    score
 Julia TRUNNER Diagonal  185 cm -0.00922
Ursula EHRHART Diagonal  185 cm -0.00922


---

Für die OR-Bedingung wurde ein `outer`-Merge verwendet und implementiert, für AND-Bedingungen ein `inner`-Merge:


In [6]:
# Suchabfrage: Test 2
# Check der OR-Bedingung
ts.search(tfidf_df, "Außenangriff or Nina", spielerinnen_csv)

volleyball_graph = og.create_volleyball_ontology()

                Name     Position Groesse    score
      Nina NESIMOVIC  Mittelblock  188 cm 0.287843
   Monika CHRTIANSKA Außenangriff  183 cm 0.188822
    Lina HINTEREGGER Außenangriff  180 cm 0.188822
Kora Marina SCHABERL Außenangriff  183 cm 0.165219


---

In [7]:
# Suchabfrage: Test 3
# Anfrage Spielerin mit zugehöriger Position, kein OR/AND

ts.search(tfidf_df, "Nicole Aufspiel", spielerinnen_csv)

volleyball_graph = og.create_volleyball_ontology()

                   Name Position Groesse   score
Nicole Leonie HOLZINGER Aufspiel  175 cm 0.20118


---

In [8]:
# Suchabfrage: Test 4
# Anfrage Spielerin ohne zugehörige Position, kein OR/AND

ts.search(tfidf_df, "Nicole Libero", spielerinnen_csv)

volleyball_graph = og.create_volleyball_ontology()

Search result: No results found.


---


Der bisherige Code aus `text_processing.py`, `tfidf_search.py` und `ontology_graph.py` wurde abgewandelt und die Ontologie eingefügt. <br>Zusätzlich wurde die Ausgabe so formatiert, dass man den Score direkt mit der jeweiligen Spielerin gemeinsam ausgegeben erhält. Die Ontologie ist zwar ab diesem Punkt bereits vorhanden, allerdings wird nach wie vor mittels Text-Token gesucht.<br>
Es ist noch nicht möglich, z.B. den Namen einer Spielerin und eine beliebige Position  opll,p]olom, p;[kklm, pl,p(0-op[l;-[p0uhtgy utsiehe **Suchabfrage Test 4**). <br>Um die Suchfunktion zu erweitern, ist **SPARQL** notwendig, da so die Ontologie-Daten tatsächlich verarbeitet werden können. Zunächst werden allerdings noch die RDF-Daten über externe Links und Quellen angereichert.

---

### 2. WIP

### 3. SPARQL Queries

Um die SPARQL Queries im Triple Storage anzuzeigen, müssen die Daten in ein Turtle Format gespeichert werden.

In [19]:
from owlready2 import *
from rdflib import Graph, Literal, RDF, RDFS, Namespace, URIRef
from rdflib.namespace import FOAF, XSD
import pandas as pd

In [20]:
graph = Graph()
# Erstellung des Volleyball Namespaces = vbn
# Beispiel: Klasse 'vbn.VolleyballPlayer' wird in RDF zu 'http://example.org/volleyball#VolleyballPlayer'
vbn = Namespace("http://example.org/volleyball#")
graph.bind("vbn", vbn)

# Classes
graph.add((vbn.Person, RDF.type, RDFS.Class))
graph.add((vbn.Athlete, RDF.type, RDFS.Class))
graph.add((vbn.VolleyballPlayer, RDF.type, RDFS.Class))
graph.add((vbn.Team, RDF.type, RDFS.Class))
graph.add((vbn.Position, RDF.type, RDFS.Class))
graph.add((vbn.Nationality, RDF.type, RDFS.Class))

# Class Hierarchies
graph.add((vbn.Athlete, RDFS.subClassOf, vbn.Person))
graph.add((vbn.VolleyballPlayer, RDFS.subClassOf, vbn.Athlete))

graph.add((vbn.hasName, RDF.type, RDF.Property))
graph.add((vbn.hasName, RDFS.domain, vbn.Person))
graph.add((vbn.hasName, RDFS.range, RDFS.Literal))

graph.add((vbn.hasPosition, RDF.type, RDF.Property))
graph.add((vbn.hasPosition, RDFS.domain, vbn.VolleyballPlayer))
graph.add((vbn.hasPosition, RDFS.range, vbn.Position))

graph.add((vbn.hasNationality, RDF.type, RDF.Property))
graph.add((vbn.hasNationality, RDFS.domain, vbn.Person))
graph.add((vbn.hasNationality, RDFS.range, vbn.Nationality))

graph.add((vbn.hasHeight, RDF.type, RDF.Property))
graph.add((vbn.hasHeight, RDFS.domain, vbn.VolleyballPlayer))
graph.add((vbn.hasHeight, RDFS.range, RDFS.Literal))

graph.add((vbn.hasJerseyNumber, RDF.type, RDF.Property))
graph.add((vbn.hasJerseyNumber, RDFS.domain, vbn.VolleyballPlayer))
graph.add((vbn.hasJerseyNumber, RDFS.range, RDFS.Literal))

graph.add((vbn.playsFor, RDF.type, RDF.Property))
graph.add((vbn.playsFor, RDFS.domain, vbn.VolleyballPlayer))
graph.add((vbn.playsFor, RDFS.range, vbn.Team))

<Graph identifier=N60b9fbfc433d4121a586162fedd9ce9f (<class 'rdflib.graph.Graph'>)>

In [21]:
df = pd.read_csv('Spielerinnendaten.csv')
graph = Graph()

graph.bind("vbn", vbn)
RDFS_NS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
graph.bind("rdfs", RDFS_NS)

team_uri = URIRef(f"http://example.org/volleyball#Nationalteam")
stadion_uri = URIRef(f"http://example.org/volleyball#ExampleStadion")

graph.add((team_uri, RDF.type, vbn.Team))
graph.add((stadion_uri, RDF.type, vbn.Stadion))

graph.add((stadion_uri, vbn.hasStadionName, Literal("Example Stadion")))
graph.add((stadion_uri, vbn.hasStadionAdress, Literal("Example Adress")))
graph.add((team_uri, vbn.hasTeamName, Literal("Nationalmannschaft")))
graph.add((team_uri, vbn.hasHomeStadion, stadion_uri))


for index, row in df.iterrows():
    player_uri = URIRef(f"http://example.org/volleyball#player_{index+1}")
    
    graph.add((player_uri, RDF.type, vbn.VolleyballPlayer))
    graph.add((player_uri, vbn.hasName, Literal(row['Name'])))
    graph.add((player_uri, vbn.hasHeight, Literal(row['Groesse'])))
    graph.add((player_uri, vbn.hasJerseyNumber, Literal(row['Dressnummer'])))
    graph.add((player_uri, vbn.playsFor, team_uri))
    
    nationality_uri = URIRef(f"http://example.org/volleyball#nationality_{row['Nationalitaet']}")
    graph.add((nationality_uri, RDF.type, vbn.Nationality))
    graph.add((player_uri, vbn.hasNationality, nationality_uri))
    
    position_uri = URIRef(f"http://example.org/volleyball#position_{row['Position']}")
    graph.add((position_uri, RDF.type, vbn.Position))
    graph.add((player_uri, vbn.hasPosition, position_uri))
    
graph.serialize(destination="volleyball_players2.ttl", format="turtle")

<Graph identifier=Nda54a377ac1b4f80bbd822a60696d1cd (<class 'rdflib.graph.Graph'>)>

1. Which players have a Number higher than 10?
2. How many players are there from each Nationality
3. How many players are there at each position on the team
4. How many players are bigger than 180 cm?
5. Name all the players on the team with their position and their numbers
6. How many players are playing on the position of "Mittelblock"?
7. Which player(s) are named Anna?
8. What is the sum of the Numbers of the team?
9. What is the average height of the players?
10. Platzhalter


Abfrage für Frage 1

PREFIX vbn: <http://example.org/volleyball#>

SELECT ?player ?name ?jerseyNumber WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasName ?name ;
          vbn:hasJerseyNumber ?jerseyNumber .
  FILTER(?jerseyNumber > 10)
}
ORDER BY ?jerseyNumber

Abfrage für Frage 2

PREFIX vbn: <http://example.org/volleyball#>

SELECT ?nationality (COUNT(?player) AS ?playerCount) WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasNationality ?nationality .
}
GROUP BY ?nationality

Gibt 15 Spielererinnen mit der Nationalität AUT zurück, wie erwartet beim Österreichischen Nationalteam

Abfrage für Frage 3: Spielerinnen pro Position


PREFIX vbn: <http://example.org/volleyball#>

SELECT ?position (COUNT(?player) AS ?playerCount) WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasPosition ?position .
}
GROUP BY ?position

Abfrage für Frage 4: Spielerinnen größer als 180 cm

PREFIX vbn: <http://example.org/volleyball#>
Prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT ?player ?name ?height WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasName ?name ;
          vbn:hasHeight ?height .
  FILTER(xsd:decimal(STRBEFORE(?height, " cm")) > 180)
}
ORDER BY DESC(?height)

Abfrage für Frage 5: Spielerinnen mit Position und Nummer

PREFIX vbn: <http://example.org/volleyball#>

SELECT ?player ?name ?position ?jerseyNumber WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasName ?name ;
          vbn:hasPosition ?position ;
          vbn:hasJerseyNumber ?jerseyNumber .
}

Abfrage für Frage 6: Spielerinnen als Mittelblock

PREFIX vbn: <http://example.org/volleyball#>

SELECT ?player ?name WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasName ?name ;
          vbn:hasPosition vbn:position_Mittelblock .
}

Abfrage für Frage 7: Größte Spielerinnen

PREFIX vbn: <http://example.org/volleyball#>
Prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT (MAX(xsd:decimal(STRBEFORE(?height, " cm"))) AS ?maxHeight) WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasHeight ?height .
}

Abfrage für Frage 8: Summe Trikotnummern

PREFIX vbn: <http://example.org/volleyball#>

SELECT (SUM(?jerseyNumber) AS ?averageJerseyNumber) WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasJerseyNumber ?jerseyNumber .
}

Abfrage für Frage 9: Durchschnittsgröße

PREFIX vbn: <http://example.org/volleyball#>
Prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT (AVG(xsd:decimal(STRBEFORE(?height, " cm"))) AS ?averageHeight) WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasHeight ?height .
}

Abfrage für Frage 10: Nach einem Namen suchen

PREFIX vbn: <http://example.org/volleyball#>

SELECT ?name
WHERE {
  ?player a vbn:VolleyballPlayer ;
          vbn:hasName ?name .
  FILTER(CONTAINS(?name, "Nicole"))
}

# Ressourcen & Source-Docs

## Inverted Index, Ontology and Search
1. [NLP Stanford - A first Take at Building an Inverted Index](https://nlp.stanford.edu/IR-book/html/htmledition/a-first-take-at-building-an-inverted-index-1.html)
2. Quasi alle vorherigen Quellen, die angegeben worden sind

## WIP
1. [Link](https://www.google.com/)
2. [Link](https://www.google.com/)
3. [Link](https://www.google.com/)
4. [Link](https://www.google.com/)
5. [Link](https://www.google.com/)

## Questions, Triple Storage
1. [Tutorial für STREBEFORE - w3 ](https://www.w3.org/TR/sparql11-query/#func-strbefore)
2. Quellen aus Aufgabe 3

# Challenges bei der Implementierung


#### Cesar Laura,

####  Dilly Julian, Fragen und Triple Storage:
Ein Teil der Schwierigkeiten war es genügend Fragen für die zehn Abfragen zu finden, da unsere Datenbasis sehr klein ist und somit nur wenige Werte enthält. Eine weitere Schwierigkeit war, dass die Werte für die Größe mit den cm gemeinsam gespeichert werden und somit die Werte als String gesehen werden. Die Lösung hierfür war es mit Hilfe der Funktion STREBEFORE diese in Integer umzuwandeln und so die richtigen Ergebnisse zu erhalten.


#### Ecker Annina, Integrierung der Onthologie in die Suche:
Grundsätzlich war unsere bestehende Logik schon sehr gut. Ich war mir nur sehr unsicher, inwiefern bereits der erste Teil "more sophisticaed" sein muss, weswegen ich mich nochmal genauer in Inverted Indices usw. eingelesen habe, und für die Suche die Abfragen zusätzlich verfeinert habe. Alles aber für Text-Token, da SPARQL semantisch deutlich bessere Abfragen machen kann, was ich gelesen habe.
